In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingRegressor, BaggingClassifier, RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from xgboost import XGBRegressor, XGBClassifier
from sklearn import preprocessing
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error, f1_score

In [2]:
# Генерируем уникальный seed
my_code = "Антонов Дмитрий"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [3]:
AD = pd.read_csv("datasets/Fish.csv")

In [4]:
AD

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340
...,...,...,...,...,...,...,...
154,Smelt,12.2,11.5,12.2,13.4,2.0904,1.3936
155,Smelt,13.4,11.7,12.4,13.5,2.4300,1.2690
156,Smelt,12.2,12.1,13.0,13.8,2.2770,1.2558
157,Smelt,19.7,13.2,14.3,15.2,2.8728,2.0672


In [5]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(AD))
print(val_test_size)

32


In [6]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(AD, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

95 32 32


In [7]:
train

,Species,Weight,Length1,Length2,Length3,Height,Width
51,Roach,180.0,23.6,25.2,27.9,7.0866,3.9060
27,Bream,714.0,32.7,36.0,41.5,16.5170,5.8515
153,Smelt,9.8,11.4,12.0,13.2,2.2044,1.1484
55,Whitefish,270.0,23.6,26.0,28.7,8.3804,4.2476
35,Roach,40.0,12.9,14.1,16.2,4.1472,2.2680
...,...,...,...,...,...,...,...
76,Perch,70.0,15.7,17.4,18.5,4.5880,2.9415
13,Bream,340.0,29.5,32.0,37.3,13.9129,5.0728
119,Perch,850.0,36.9,40.0,42.3,11.9286,7.1064
52,Roach,290.0,24.0,26.0,29.2,8.8768,4.4968


In [8]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
num_columns = ['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), num_columns)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['Weight', 'Length1', 'Length2', 'Length3',
                                  'Height', 'Width'])])

In [9]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [10]:
sc_train 

,0,1,2,3,4,5,6
0,0.105459,0.287726,0.287313,0.298951,0.328553,0.436517,Roach
1,0.430414,0.470825,0.488806,0.536713,0.906803,0.733621,Bream
2,0.001886,0.042254,0.041045,0.041958,0.029187,0.015394,Smelt
3,0.160226,0.287726,0.302239,0.312937,0.407885,0.488684,Whitefish
4,0.020264,0.072435,0.080224,0.094406,0.148315,0.186372,Roach
...,...,...,...,...,...,...,...
90,0.03852,0.128773,0.141791,0.134615,0.175344,0.289225,Perch
91,0.202824,0.406439,0.414179,0.463287,0.747126,0.614703,Bream
92,0.513175,0.555332,0.563433,0.550699,0.625453,0.925262,Perch
93,0.172397,0.295775,0.302239,0.321678,0.438324,0.52674,Roach


In [11]:
# Задание №1 - анализ различных типов ансамблей решений в задаче регрессии
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html
# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn

In [12]:
# Устанавливаем названия столбцов
column_names = num_columns + ['Species']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [13]:
# Явно укажем типы данных, это важно для xgboost
types = {
    'Weight' : 'float64',
    'Length1' : 'float64',
    'Length2' : 'float64',
    'Length3' : 'float64',
    'Height' : 'float64',
    'Width' : 'float64',
    'Species' : 'category'
}
sc_train = sc_train.astype(types)
sc_test = sc_test.astype(types)
sc_val = sc_val.astype(types)

In [14]:
# Выбираем 4 числовых переменных, три их них будут предикторами, одна - зависимой переменной
n = 4
labels = random.sample(num_columns, n)

y_labels = labels[0]
x_labels = labels[1:]

print(x_labels)
print(y_labels)

['Weight', 'Length2', 'Length3']
Width


In [15]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_labels]
y_test = sc_test[y_labels]
y_val = sc_val[y_labels]

In [16]:
# Создайте 4 различных модели с использованием следующих классов:
# BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor, XGBRegressor.
# Решите получившуюся задачу регрессии с помощью созданных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

# mean_squared_error -> min 

In [17]:
# Общий список моделей
r_models=[BaggingRegressor(), RandomForestRegressor(),GradientBoostingRegressor(), XGBRegressor()]

In [18]:
r_models=[]

#Регрессор Бэгинга
r_models.append(BaggingRegressor())
#случайный лес
r_models.append(RandomForestRegressor())
#градиентный бустинг
r_models.append(GradientBoostingRegressor())

#XGBregre
r_models.append(XGBRegressor())

In [19]:
r_models

[BaggingRegressor(),
 RandomForestRegressor(),
 GradientBoostingRegressor(),
 XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None)]

In [20]:
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [21]:
y_train

array([0.43651691, 0.73362145, 0.01539354, 0.48868391, 0.18637183,
       0.75675758, 0.94337375, 0.28565102, 0.62966922, 0.48737058,
       0.58770349, 0.84389603, 0.44735958, 0.42414709, 0.49754131,
       0.34511774, 0.97261843, 0.61949849, 0.01539354, 0.34727101,
       0.97962799, 0.15570691, 0.77624385, 0.45010843, 0.37833298,
       0.88891604, 0.57973183, 0.40009468, 0.98231575, 0.55624446,
       0.37833298, 0.64636083, 0.19530558, 0.24822089, 0.38291439,
       0.57527259, 0.78244403, 0.1934272 , 0.66015088, 0.4025381 ,
       0.42032925, 0.031795  , 0.81547601, 0.8882441 , 0.3797074 ,
       0.03628478, 0.32482209, 0.36083198, 0.39455117, 0.56901133,
       0.05076204, 0.62337742, 0.42570477, 0.21141688, 0.62455331,
       0.69055618, 0.80307565, 0.81287988, 0.35551755, 0.71470022,
       0.5204331 , 0.48882136, 0.50012217, 0.26333954, 0.5924987 ,
       0.47500076, 0.01806603, 0.28355884, 0.42342934, 0.        ,
       0.85250909, 0.47286277, 0.95134541, 0.45392627, 0.50203

In [22]:
for model in r_models:
    print(model)
    model.fit(x_train, y_train)

BaggingRegressor()
RandomForestRegressor()
GradientBoostingRegressor()
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None,
             enable_categorical=False, gamma=None, gpu_id=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_delta_step=None, max_depth=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=None)


In [23]:
# Оценииваем качество работы моделей на валидационной выборке.
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(model, '\t', mse)

BaggingRegressor() 	 0.0028379807612619965
RandomForestRegressor() 	 0.0023090859087358346
GradientBoostingRegressor() 	 0.0025369413476276125
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None) 	 0.003068263856697519


In [24]:
# Выбираем лучшую модель
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

RandomForestRegressor()

In [25]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.005351377644829736


In [26]:
# Задание №2 - анализ различных типов ансамблей в задаче классификации
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn

In [27]:
n = 2
x_labels = random.sample(num_columns, n)
y_label = 'Species'

print(x_labels)
print(y_label)

['Length1', 'Length2']
Species


In [28]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [29]:
y_train

0         Roach
1         Bream
2         Smelt
3     Whitefish
4         Roach
        ...    
90        Perch
91        Bream
92        Perch
93        Roach
94         Pike
Name: Species, Length: 95, dtype: category
Categories (7, object): ['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish']

In [30]:
# Создайте 4 различных модели с использованием следующих классов:
# BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, XGBClassifier
# Решите получившуюся задачу классификации с помощью созданных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

# f1_score -> max

In [31]:
#Преобразовываем метки классов в целые числа
le = preprocessing.LabelEncoder()
y_test


0         Perch
1         Bream
2         Bream
3        Parkki
4         Bream
5         Perch
6         Bream
7         Bream
8          Pike
9         Perch
10        Roach
11        Perch
12         Pike
13        Perch
14         Pike
15        Bream
16        Smelt
17        Roach
18        Perch
19        Roach
20        Perch
21        Smelt
22        Bream
23        Roach
24        Perch
25    Whitefish
26        Roach
27        Roach
28        Perch
29        Perch
30        Smelt
31        Perch
Name: Species, dtype: category
Categories (7, object): ['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish']

In [32]:
le.fit(y_train)
le.fit(y_val)
le.fit(y_test)

LabelEncoder()

In [33]:
y_train=le.transform(y_train)
y_val=le.transform(y_val)
y_test=le.transform(y_test)

In [34]:
y_train

array([4, 0, 5, 6, 4, 2, 2, 1, 0, 2, 3, 6, 3, 2, 0, 2, 2, 0, 5, 2, 2, 5,
       0, 2, 2, 2, 0, 2, 3, 0, 2, 0, 1, 1, 2, 0, 3, 1, 0, 2, 4, 5, 2, 2,
       2, 5, 4, 1, 4, 4, 5, 3, 4, 2, 2, 0, 0, 0, 3, 0, 0, 6, 3, 2, 0, 3,
       5, 4, 4, 5, 2, 1, 2, 0, 2, 2, 1, 3, 0, 0, 2, 6, 2, 2, 3, 0, 0, 2,
       2, 4, 2, 0, 2, 4, 3])

In [35]:
a_models=[BaggingClassifier(),     
          RandomForestClassifier(),    
          GradientBoostingClassifier(),    
          XGBClassifier()]

In [36]:
a_models=[]

#Регрессор Бэгинга
a_models.append(BaggingClassifier())

#случайный лес
a_models.append(RandomForestClassifier())

#градиентный бустинг
a_models.append(GradientBoostingClassifier())

#XGBreg
a_models.append(XGBClassifier(use_label_encoder=False,
                              eval_metric='mlogloss'))


In [37]:
# Обучаем модели
for model in a_models:
    print(model)
    model.fit(x_train, y_train)

BaggingClassifier()
RandomForestClassifier()
GradientBoostingClassifier()
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, eval_metric='mlogloss', gamma=None,
              gpu_id=None, importance_type=None, interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, use_label_encoder=False,
              validate_parameters=None, verbosity=None)


In [38]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in a_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(model, '\t', f1)

BaggingClassifier() 	 0.552639751552795
RandomForestClassifier() 	 0.552639751552795
GradientBoostingClassifier() 	 0.552639751552795
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='mlogloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', use_label_encoder=False,
              validate_parameters=1, ...) 	 0.49032738095238093


In [39]:
# Выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_a_model = a_models[i_max]
best_a_model

BaggingClassifier()

In [40]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_a_model.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.45575657894736843
